In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# URL of the page to scrape
url = "https://fbref.com/en/matches/46d9048f/"

# Get the game ID from the URL
game_id = url.split("/")[-2]

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the Shots table on the page
shots_table = soup.find("div", {"id": "switcher_shots"})

# Create empty lists to store data and create the columns
data = []
headers = ['Game_ID', 'Minute', 'Player', 'Team', 'xG', 'PSxG', 'Result', 'Distance', 'Body Part', 'Notes', 'SCA 1 Player', 'Event', 'SCA 2 Player', 'Event']

# Extract table rows
rows = shots_table.find("tbody").find_all("tr")
for row in rows:
    row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
    data.append(row_data)

# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(data, columns=headers)

# Find the home and away teams and formations
team_headers = soup.find_all("th", {"colspan": "2"})
teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

# Extracting the home and away teams and formations
home_team = teams[0]
away_team = teams[1]
home_formation = formations[0]
away_formation = formations[1]

# Find the competition -- only using .find because there are multiple in the HTML and we only need one, then take only the text
competition = soup.find("a", {"href": "/en/comps/10/2022-2023/2022-2023-Championship-Stats"}).text

# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(data, columns=headers)

# Add the team, away team, formation, and opponent formation information to the DataFrame
shots_df['Home_Team'] = home_team
shots_df['Away_Team'] = away_team
shots_df['Home Formation'] = home_formation
shots_df['Away Formation'] = away_formation
shots_df['is_home_shot'] = np.where(shots_df['Team'] == home_team, 1, 0)
shots_df['is_away_shot'] = np.where(shots_df['Team'] ==  away_team, 1, 0)

In [17]:
# Find the span element with class "venuetime"
span_venuetime = soup.find("span", class_="venuetime")

# Extract the data-venue-date attribute
match_date = span_venuetime.get("data-venue-date") if span_venuetime else None

# Print the venue date
print(match_date)


2022-07-29


In [43]:
output = soup.find("div", {"class": "scorebox_meta"}).find("small").text

KeyError: 3

In [50]:
output = soup.find("div", {"class": "scorebox_meta"}).find_all("small")[3].text

In [51]:
output

<small>The John Smith's Stadium, Huddersfield</small>